importing all the dependencies first

In [5]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10

DATASET:
- 50k train images
- 10k test images
- 32*32 pixels RGB (meaning 3 channels)

In [6]:
# loading the dataset
(x_train,y_train), (x_test,y_test) = cifar10.load_data()
# normalize and convert type
x_train = x_train.astype("float32")/255.0
x_test = x_test.astype("float32")/255.0

- as we are using CNN, and we want spatial information to be preserved, we dont want to flatten our input.
- thus we can keep our input shape to (32,32,3)

#### Sequential Model

Building the model

In [7]:
model = keras.Sequential(
    [
        keras.Input(shape=(32,32,3)),
        layers.Conv2D(32,3,padding='valid',activation='relu'),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Conv2D(64,3,activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(128,3,activation='relu'),

        layers.Flatten(),
        layers.Dense(64,activation='relu'),
        layers.Dense(10)
    ]
)

In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 15, 15, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 flatten_1 (Flatten)         (None, 2048)             

Compiling the model

In [11]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(learning_rate=3e-4),
    metrics = ['accuracy']
)

Training the model

In [12]:
model.fit(x_train,y_train,batch_size=64,epochs=10,verbose=2)

Epoch 1/10


2022-05-04 11:23:00.910470: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 614400000 exceeds 10% of free system memory.


782/782 - 17s - loss: 1.6796 - accuracy: 0.3885 - 17s/epoch - 22ms/step
Epoch 2/10
782/782 - 17s - loss: 1.3428 - accuracy: 0.5206 - 17s/epoch - 22ms/step
Epoch 3/10
782/782 - 18s - loss: 1.2153 - accuracy: 0.5701 - 18s/epoch - 22ms/step
Epoch 4/10
782/782 - 19s - loss: 1.1216 - accuracy: 0.6071 - 19s/epoch - 25ms/step
Epoch 5/10
782/782 - 20s - loss: 1.0492 - accuracy: 0.6325 - 20s/epoch - 26ms/step
Epoch 6/10
782/782 - 18s - loss: 0.9901 - accuracy: 0.6538 - 18s/epoch - 23ms/step
Epoch 7/10
782/782 - 17s - loss: 0.9327 - accuracy: 0.6759 - 17s/epoch - 22ms/step
Epoch 8/10
782/782 - 17s - loss: 0.8909 - accuracy: 0.6914 - 17s/epoch - 22ms/step
Epoch 9/10
782/782 - 17s - loss: 0.8518 - accuracy: 0.7048 - 17s/epoch - 22ms/step
Epoch 10/10
782/782 - 17s - loss: 0.8116 - accuracy: 0.7206 - 17s/epoch - 21ms/step
